# Basic Weaviate operations

This notebook uses standalone cells which can be executed independently of each other. 

The code is designed to connect to a local Weaviate instance running with default port configuration. If you prefer to use the free cloud sandbox (which is deleted after 14 days), replace any instance of client with the code below. Make sure that the environment variables WCD_URL (REST url of the sandbox instance) and WCD_API_KEY are available in your environment. 


In [ ]:
# The code to connect to a managed cloud weaviate instance is as follows:

# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=os.getenv("WCD_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WCD_API_KEY")),
#     headers={
#         "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY_RAGPOC"]  # Replace with your inference API key
#     }
# )

## Load data to the Weaviate instance

In [ ]:
import weaviate
import weaviate.classes as wvc
import os
import requests
import json

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY_RAGPOC")
    }
)

try:
    # Delete the collection in case it already exists to prevent a "collection already exists" error
    client.collections.delete("Questions")
    
    # ===== define collection =====
    questions = client.collections.create(
        name="Questions",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
        generative_config=wvc.config.Configure.Generative.openai()  # Ensure the `generative-openai` module is used for generative queries
    )

    # ===== import data =====
    resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
    data = json.loads(resp.text)  # Load data

    question_objs = list()
    for i, d in enumerate(data):
        question_objs.append({
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        })

    questions = client.collections.get("Questions")
    questions.data.insert_many(question_objs)
    
finally:
    client.close()  # Close client gracefully

## Query the Weaviate instance

### Print the vectors for all objects in the collection

In [ ]:
import weaviate
import weaviate.classes as wvc
import os

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY_RAGPOC")
    }
)

try:
    questions = client.collections.get("Questions")
    response = questions.query.fetch_objects(include_vector=True)
    for o in response.objects:
        print(o.vector)

finally:
    client.close()  # Close client gracefully

### Semantic search demo

In [ ]:
import weaviate
import weaviate.classes as wvc
import os

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY_RAGPOC")
    }
)

try:
    questions = client.collections.get("Questions")

    response = questions.query.near_text(
        query="biology",
        limit=2
    )

    print(response.objects[0].properties)  # Inspect the first object

finally:
    client.close()  # Close client gracefully

### Semantic search with a filter

In [ ]:
import weaviate
import weaviate.classes as wvc
import os

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY_RAGPOC")
    }
)
try:
    questions = client.collections.get("Questions")

    response = questions.query.near_text(
        query="biology",
        limit=2,
        filters=wvc.query.Filter.by_property("category").equal("ANIMALS")
    )

    print(response.objects[0].properties)  # Inspect the first object

finally:
    client.close()  # Close client gracefully

### Generative search

In [ ]:
import weaviate
import weaviate.classes as wvc
import os

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY_RAGPOC")
    }
)

try:
    questions = client.collections.get("Questions")

    response = questions.generate.near_text(
        query="biology",
        limit=2,
        single_prompt="Write a limerick in only two lines about {answer}."
    )

    print(response.objects[0].generated)  # Inspect the first object

finally:
    client.close()  # Close client gracefully

### Generative search (grouped task)

In [ ]:
import weaviate
import weaviate.classes as wvc
import os

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY_RAGPOC")
    }
)

try:
    questions = client.collections.get("Questions")

    response = questions.generate.near_text(
        query="biology",
        limit=2,
        grouped_task="Write a tweet combining those topics"
    )

    print(response.generated)  # Inspect the first object

finally:
    client.close()  # Close client gracefully